## import

In [148]:
import numpy as np
import sys,os
import random
import matplotlib.pyplot as plt
import csv
import ast
from collections import OrderedDict

## Input Data 생성

x, y, z 데이터 중 

x가 0~99이면    정답 => [1,0,0]

x가 100~199이면 정답 => [0,1,0]

x가 200~299이면 정답 => [0,0,1]

In [149]:
input_data=np.empty((0,3))
t=np.empty((0,3))
for i in range(1000):
    input_data=np.append(input_data, np.array([[random.randrange(0,300) for j in range(3)]]),axis=0)
    x=input_data[-1,0]
    if x<100:
        t=np.append(t,np.array([[1, 0, 0]]),axis=0)
    elif x<200:
        t=np.append(t,np.array([[0, 1, 0]]),axis=0)
    elif x<300:
        t=np.append(t,np.array([[0, 0, 1]]),axis=0)
print(input_data.shape)

(1000, 3)


## Input Data Normalize  (0~1)

정규화 식


<img src = "../simple_network/Data/normalize.png" width = "200" height = "75">



In [150]:
input_max=299
input_min=0
de=input_max-input_min
normalize_input=np.zeros((1000,3),dtype=float)
for i in range(1000):
    for j in range(3):
        normalize_input[i,j]=(input_data[i,j]-input_min)/de
print(input_data)
print(normalize_input)

[[157. 235. 255.]
 [250.  24. 200.]
 [133.  60. 106.]
 ...
 [ 79. 282. 164.]
 [167. 256. 158.]
 [220. 289.  48.]]
[[0.52508361 0.78595318 0.85284281]
 [0.8361204  0.08026756 0.66889632]
 [0.44481605 0.2006689  0.35451505]
 ...
 [0.26421405 0.94314381 0.54849498]
 [0.55852843 0.85618729 0.52842809]
 [0.73578595 0.96655518 0.16053512]]


### Input Data 파일에 저장

In [151]:
np.savez("../simple_network/Data/input_data_set.npz", input_data)
np.savez("../simple_network/Data/t_set.npz", t)

#### Input Data 파일 읽기 테스트

np.savez()를 사용하여 numpy array 형 변수 읽어오기


In [152]:
read_input=np.load("../simple_network/Data/input_data_set.npz", allow_pickle=True)
read_t=np.load("../simple_network/Data/t_set.npz", allow_pickle=True)
r_input_data=read_input["arr_0"]
r_t_data=read_t["arr_0"]
print(r_input_data)
print(r_t_data)

[[157. 235. 255.]
 [250.  24. 200.]
 [133.  60. 106.]
 ...
 [ 79. 282. 164.]
 [167. 256. 158.]
 [220. 289.  48.]]
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## 가중치 W, 편향 B  생성 ##
정규분포를 따르는 랜덤수로 생성

0.01을 곱해주는 이유: w의 초기값이크면 gradient vanishing 문제가 발생할 수 있음.


In [153]:
weight_init_std=0.01
params={}
params['W1']=weight_init_std*np.random.randn(3,4)
params['B1']=np.zeros(4)
params['W2']=weight_init_std*np.random.randn(4,4)
params['B2']=np.zeros(4)
params['W3']=weight_init_std*np.random.randn(4,3)
params['B3']=np.zeros(3)

## 가중치 W, 편향 B weight_bias.npz 파일에 저장

np.savez() --> 여러개의 배열을 1개의 압축되지 않은 *.npz 포맷 파일로 저장하기 (Save several arrays into a single file in uncompressed .npz format)

In [154]:
np.savez("../simple_network/Data/weight_bias.npz", **params)

##### dictionary형 변수 csv파일에 저장

In [155]:
# open a file in write mode
with open('../simple_network/Data/weight_bias.csv', 'w', newline='') as f:
    # create a csv writer
    writer = csv.writer(f)
    # write the keys of the dictionary as the header row
    writer.writerow(params.keys())
    # write the values of the dictionary as a single row
    writer.writerow(params.values())


npz 변수 파일 불러오기

np.load("파일명",allow_pickle=True)

In [156]:
my_dictionary = np.load("../simple_network/Data/weight_bias.npz", allow_pickle=True)
paaa=dict(my_dictionary)
print(paaa)
print(params)

{'W1': array([[-0.00964063, -0.01990779,  0.00089731,  0.00114854],
       [-0.00585815,  0.00298772,  0.0022226 ,  0.00435183],
       [-0.00045748,  0.00049898, -0.0093553 ,  0.00287388]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[ 0.00360427,  0.00408148, -0.01940716,  0.01444836],
       [ 0.00192861, -0.00420865,  0.01740253, -0.00364087],
       [ 0.01343954, -0.00818221,  0.0008271 , -0.01291058],
       [-0.00661104, -0.01180191,  0.00197643,  0.004139  ]]), 'B2': array([0., 0., 0., 0.]), 'W3': array([[ 0.01197322,  0.01883354,  0.00714224],
       [ 0.02284333,  0.01564103,  0.00611104],
       [-0.00877363, -0.01621087, -0.00581673],
       [-0.00537834, -0.01556024, -0.00054465]]), 'B3': array([0., 0., 0.])}
{'W1': array([[-0.00964063, -0.01990779,  0.00089731,  0.00114854],
       [-0.00585815,  0.00298772,  0.0022226 ,  0.00435183],
       [-0.00045748,  0.00049898, -0.0093553 ,  0.00287388]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[ 0.00360427,  0.00408148, -0.0